# Dance Entropy

## Imports and Variables

In [359]:
import pandas as pd
from scipy.stats import zscore # normalize data. Not used.
import scipy.signal as signal
import matplotlib.pyplot as plt
import numpy as np
import sampen
import glob
import os

from sampen import sampen2
    # https://github.com/bergantine/sampen/
from pyentrp import entropy as ent
    # https://github.com/nikdon/pyEntropy 
    
import sys
sys.path.insert(0, 'C:\\Users\\albei\\entropy\\')
from entropy import *
    # https://github.com/raphaelvallat/entropy

In [360]:
variable = "Joint"
path = 'output\\'+variable
all_xlsx = glob.glob(os.path.join(path, "*.xlsx"))  # make a list of paths
shape = len(all_xlsx)

## CSV Converter

In [362]:
all_csv = glob.glob(os.path.join(path, "*.csv"))  # make a list of paths
all_csv
i = 0
shape = len(all_csv)

for files in all_csv:
  dir_path = os.path.dirname(os.path.realpath(files)) # get the csv file's current directory
  csv_file = os.path.splitext(os.path.basename(files))[0]  # get file name without extension
  xlsx_file = dir_path + "\\"+ csv_file + ".xlsx" # save csv file in current directory
  pd.read_csv(files, delimiter= ",").to_excel(xlsx_file, index=False)
  i = i + 1
  print("||",i, "/", shape, " FINISHED! || ", csv_file)


|| 1 / 11  FINISHED! ||  2018-07-14-12-32_pddance001_Tango - Part 2 (harder)
|| 2 / 11  FINISHED! ||  2018-08-11-11-48_pddance002_rumba1
|| 3 / 11  FINISHED! ||  2018-08-11-11-55_pddance002_line
|| 4 / 11  FINISHED! ||  2018-08-11-12-03_pddance002_foxtrot1
|| 5 / 11  FINISHED! ||  2018-08-11-12-08_pddance002_foxtrot2
|| 6 / 11  FINISHED! ||  2018-08-11-12-13_pddance002_waltzprogressive
|| 7 / 11  FINISHED! ||  2018-08-11-12-19_pddance002_tango1
|| 8 / 11  FINISHED! ||  2018-08-11-12-20_pddance002_tango2
|| 9 / 11  FINISHED! ||  2018-08-11-12-30_pddance002_sidetango
|| 10 / 11  FINISHED! ||  2018-10-13-12-11_pddance003_American Tango
|| 11 / 11  FINISHED! ||  2018-10-13-12-26_pddance003_Tango and Corte


## Calculate Approximate Entropy using EntroPy

* perm_entropy(x, order=3, normalize=True)                 # Permutation entropy
* spectral_entropy(x, 100, method='welch', normalize=True) # Spectral entropy
* svd_entropy(x, order=3, delay=1, normalize=True)         # Singular value decomposition entropy
* app_entropy(x, order=2, metric='chebyshev')              # Approximate entropy
* sample_entropy(x, order=2, metric='chebyshev')           # Sample entropy
* lziv_complexity('01111000011001', normalize=True)        # Lempel-Ziv complexity


In [353]:
def dance_appen_calc(all_xlsx):
    ''' Calculates the approximate entropy for acceleration vectors only. No zscore.
    
    Function to calculate entropy for all files in a list, 
    save it as a dictionary, convert to a dataframe, and format it for easy reading.
    Designed for KinesiaONE tremor raw data
    
    appen_calc(file_list)
    '''
    i = 0
    output = variable + "_appen_dance.xlsx"
    file_dict = {}
    for file in all_xlsx:
        i = i + 1
        df = pd.read_excel(file)
        df = df.apply(zscore) # normalize data by getting zscores
        filename = file.split(sep='\\')[-1].split(sep=".")[0] # get the filename
        datetime = filename.split("_")[0]
        participant = filename.split(sep="_")[1]
        dance = filename.split(sep="_")[2]
        cols = df.columns
        apen_dict = {}
        apen_dict['participant'] = participant
        apen_dict["date"] = datetime
        apen_dict["dance"] = dance
        for col in cols:
            apen = app_entropy(np.array(df[col]), order=2, metric='chebyshev')
            apen_dict[col] = apen
        file_dict[filename] = apen_dict
        print('||', i, "/", shape, "Finished! || ", filename)
    apen_df = pd.DataFrame(file_dict).reset_index()
    #apen_df = apen_df.sort_values("index").pivot_table(columns="index", aggfunc="first")
    #apen_df['date'] = pd.to_datetime(apen_df['date'], format='%Y-%m-%d-%H-%M')
    #apen_df = apen_df.reindex(columns = sorted(joint_apen.columns,reverse=True)).reset_index()
    #apen_df.to_excel(output, index=False)
    return apen_df

In [354]:
joint_apen = dance_appen_calc(all_xlsx)

|| 1 / 9 Finished! ||  2018-07-14-12-32_pddance001_Tango - Part 2 (harder)
|| 2 / 9 Finished! ||  2018-08-11-11-48_pddance002_rumba1
|| 3 / 9 Finished! ||  2018-08-11-11-55_pddance002_line
|| 4 / 9 Finished! ||  2018-08-11-12-03_pddance002_foxtrot1
|| 5 / 9 Finished! ||  2018-08-11-12-08_pddance002_foxtrot2
|| 6 / 9 Finished! ||  2018-08-11-12-13_pddance002_waltzprogressive
|| 7 / 9 Finished! ||  2018-08-11-12-19_pddance002_tango1
|| 8 / 9 Finished! ||  2018-08-11-12-20_pddance002_tango2
|| 9 / 9 Finished! ||  2018-08-11-12-30_pddance002_sidetango


In [358]:
joint_apen.to_excel("Course_appen_dance.xlsx",index=False)